## Importing Data and Initial Cleaning

In [156]:
import os
import pandas as pd
import pickle

In [251]:
# Initialize Dataframe 

df = pd.DataFrame(columns=['president','address'])

In [252]:
# Designate Path

path = 'C:\\Users\\Andrew\\Documents\\Metis\\NLP_Inaugural_Addresses\\Data\\InauguralTexts'

In [253]:
# Read all .txt files and append to a single dataframe

for filename in os.listdir(path):
    with open(os.path.join(path,filename)) as f:
        address = f.read()
        current_df = pd.DataFrame({'address': [address],'president':[filename]})
        df = df.append(current_df, ignore_index=True)

In [254]:
# Renaming "Roosevelt" to "FRoosevelt"

for i in range(36,40):
    df['president'][i] = df['president'][i].replace("Roosevelt","FRoosevelt")

# Renaming "Roosevelt" to "TRoosevelt"
df['president'][29] = df['president'][29].replace("Roosevelt","TRoosevelt")

In [255]:
# Creating a Column that Represents President Number

df['president_number'] = [i.strip(".")[0:2] for i in df.president]
df = df[['president_number','president','address']]

In [256]:
# Creating a column that is only the presidents name

df['pres_name'] = [i.split(".")[1] for i in df['president']]

# Creat a column that represents term number

df['term'] = [i.split(".")[2] for i in df['president']]

In [257]:
# re-orient columns
df = df[['president_number','term','pres_name','president','address']]

In [258]:
# Merging Party Information
path = r"C:\Users\Andrew\Documents\Metis\NLP_Inaugural_Addresses\Data\PresidentInfo"
df_party = pd.read_csv(path+"\Pres_Details.csv")

In [259]:
# Convert to int for merge
df['president_number'] = df['president_number'].astype(int)

# Perform Merge
df = df.merge(df_party,left_on="president_number",right_on='S.No.')

# Drop Columns
df = df.drop(['start','end','president_y','vice','prior','S.No.'],axis=1)

In [260]:
# Merging year of first term
path = r"C:\Users\Andrew\Documents\Metis\NLP_Inaugural_Addresses\Data\PresidentInfo"
df_dates = pd.read_csv(path+"\Inaugural_Dates.csv")

In [261]:
# Perform Merge
df = pd.concat([df,df_dates[['Year','Pres']]],axis=1).drop('Pres',axis=1)
df.head()

,president_number,term,pres_name,president_x,address,party,Year
0,1,1,Washington,01.Washington.1.txt,AMONG the vicissitudes incident to life no eve...,Nonpartisan,1789
1,1,2,Washington,01.Washington.2.txt,I AM again called upon by the voice of my coun...,Nonpartisan,1793
2,2,1,Adams,02.Adams.1.txt,"WHEN it was first perceived, in early times, ...",Federalist,1797
3,3,1,Jefferson,03.Jefferson.1.txt,CALLED upon to undertake the duties of the fi...,Democratic-Republican,1801
4,3,2,Jefferson,03.Jefferson.2.txt,"PROCEEDING, fellow-citizens, to that qualific...",Democratic-Republican,1805


In [271]:
# Create helper function to group each address into 50 year blocks

def time_period(x):
    if x < 1801:
        period = 'pre-1800'
    if x > 1800 and i < 1851:
        period = '1800-1850'
    if x > 1850 and i < 1901:
        period = '1851-1900'
    if x > 1900 and i < 1951:
        period = '1901-1950'
    if x > 1950 and i <2001:
        period = '1951-2000'
    if x > 2000:
        period = 'post-2000'
    return period 

In [273]:
# Apply helper function to each row

df['time_period'] = df['Year'].apply(lambda x: time_period(x))
df.head()

,president_number,term,pres_name,president_x,address,party,Year,time_period
0,1,1,Washington,01.Washington.1.txt,AMONG the vicissitudes incident to life no eve...,Nonpartisan,1789,pre-1800
1,1,2,Washington,01.Washington.2.txt,I AM again called upon by the voice of my coun...,Nonpartisan,1793,pre-1800
2,2,1,Adams,02.Adams.1.txt,"WHEN it was first perceived, in early times, ...",Federalist,1797,pre-1800
3,3,1,Jefferson,03.Jefferson.1.txt,CALLED upon to undertake the duties of the fi...,Democratic-Republican,1801,1800-1850
4,3,2,Jefferson,03.Jefferson.2.txt,"PROCEEDING, fellow-citizens, to that qualific...",Democratic-Republican,1805,1800-1850


In [279]:
# Group Presidents by Party (seperate pre-Lincoln Dems)

# first remove extra space in republican
df['party'] = df['party'].apply(lambda x: str.strip(x)) 

In [282]:
# Pickle dataframe and save for later use

path = r"C:\Users\Andrew\Documents\Metis\NLP_Inaugural_Addresses\Pickled_Files\cleaned_addresses.pkl"
with open(path, 'wb') as f:
    pickle.dump(df, f)

In [283]:
df.head(10)

,president_number,term,pres_name,president_x,address,party,Year,time_period
0,1,1,Washington,01.Washington.1.txt,AMONG the vicissitudes incident to life no eve...,Nonpartisan,1789,pre-1800
1,1,2,Washington,01.Washington.2.txt,I AM again called upon by the voice of my coun...,Nonpartisan,1793,pre-1800
2,2,1,Adams,02.Adams.1.txt,"WHEN it was first perceived, in early times, ...",Federalist,1797,pre-1800
3,3,1,Jefferson,03.Jefferson.1.txt,CALLED upon to undertake the duties of the fi...,Democratic-Republican,1801,1800-1850
4,3,2,Jefferson,03.Jefferson.2.txt,"PROCEEDING, fellow-citizens, to that qualific...",Democratic-Republican,1805,1800-1850
5,4,1,Madison,04.Madison.1.txt,UNWILLING to depart from examples of the most...,Democratic-Republican,1809,1800-1850
6,4,2,Madison,04.Madison.2.txt,ABOUT to add the solemnity of an oath to the o...,Democratic-Republican,1813,1800-1850
7,5,1,Monroe,05.Monroe.1.txt,I SHOULD be destitute of feeling if I was not ...,Democratic-Republican,1817,1800-1850
8,5,2,Monroe,05.Monroe.2.txt,I SHALL not attempt to describe the grateful...,Democratic-Republican,1821,1800-1850
9,6,1,Adams,06.Adams.1.txt,IN compliance with an usage coeval with the e...,Democratic-Republican,1825,1800-1850
